# Unit Cube

This Nootebook have the goal to calculate the total energy inside a hexahedric element
applying the continum mechanics equations [Bonet 1997]

In [2]:
# Libraries 
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
import pyvista as pv
from jax import random
import jax
key = random.key(0)

2024-04-05 18:05:02.340360: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.0 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [3]:
## import mesh

mesh = pv.read('/home/nicolas/Escritorio/Codigos/JAX/Unit_Tests/unit_cube.msh')  #Orgiinal File
mesh.clear_data()

## displacement from FEM software Vulcan
disp = np.load('/home/nicolas/Escritorio/Codigos/JAX/Unit_Tests/cube_displacements.npy')[-1,:,:]  # The orden of this file is [time, node, dim]
disp_tot = np.load('/home/nicolas/Escritorio/Codigos/JAX/Unit_Tests/cube_displacements.npy')
mesh_def = mesh.copy()
mesh_def.points += disp

pl = pv.Plotter(shape=(1, 2))


pl.subplot(0, 0)
pl.add_text("Original State", font_size=30)
pl.add_mesh(mesh, show_edges=True, color='lightblue',opacity = 1)


pl.subplot(0, 1)
pl.add_text("Deformed State", font_size=30)
pl.add_mesh(mesh_def, show_edges=True, color='lightblue')


# Display the window
pl.show()

Widget(value="<iframe src='http://localhost:35233/index.html?ui=P_0x7fdde4322b30_0&reconnect=auto' style='widt…

Work line is:

displacement -> deformation gradient -> Energy -> Total Energy Integral



In [7]:
class energy():

    def __init__(self, nodes, conn):
        self.conn = conn
        self.nodes = nodes[conn]
        self.nnodes = 8


        puntos_iso = jnp.array([[-1,-1,-1],
                               [ 1,-1,-1],
                               [ 1, 1,-1],
                               [-1, 1,-1],
                               [-1,-1, 1],
                               [ 1,-1, 1],
                               [ 1, 1, 1],
                               [-1, 1, 1] ])
        

        self.gauss_points = jnp.array([[-1/3**0.5, -1/3**0.5, -1/3**0.5],
                                      [ 1/3**0.5, -1/3**0.5, -1/3**0.5],
                                      [ 1/3**0.5,  1/3**0.5, -1/3**0.5],
                                      [-1/3**0.5,  1/3**0.5, -1/3**0.5],
                                      [-1/3**0.5, -1/3**0.5,  1/3**0.5],
                                      [ 1/3**0.5, -1/3**0.5,  1/3**0.5],
                                      [ 1/3**0.5,  1/3**0.5,  1/3**0.5],
                                      [-1/3**0.5,  1/3**0.5,  1/3**0.5],
                                      ])
        
        self.der_N_X_gp = [self.der_N_X(i) for i in self.gauss_points]
        self.der_N_X_gp = jnp.array(self.der_N_X_gp).transpose((1,0,2,3))
        
        self.der_N_X_esquinas = [self.der_N_X(i) for i in puntos_iso]
        self.der_N_X_esquinas = np.array(self.der_N_X_esquinas).transpose((1,0,2,3))


    def _get_nodes(self, x):
        return x[self.conn,:]


    def N_func(self, xi):
        xi0 = xi[0]
        xi1 = xi[1]
        xi2 = xi[2]
        N1 = (1.0 - xi0)*(1.0 - xi1)*(1.0 - xi2)/8.0  
        N2 = (1.0 + xi0)*(1.0 - xi1)*(1.0 - xi2)/8.0  
        N3 = (1.0 + xi0)*(1.0 + xi1)*(1.0 - xi2)/8.0  
        N4 = (1.0 - xi0)*(1.0 + xi1)*(1.0 - xi2)/8.0  
        N5 = (1.0 - xi0)*(1.0 - xi1)*(1.0 + xi2)/8.0  
        N6 = (1.0 + xi0)*(1.0 - xi1)*(1.0 + xi2)/8.0  
        N7 = (1.0 + xi0)*(1.0 + xi1)*(1.0 + xi2)/8.0
        N8 = (1.0 - xi0)*(1.0 + xi1)*(1.0 + xi2)/8.0
        return jnp.array([N1, N2, N3, N4, N5, N6, N7, N8])

    def der_N_fun(self, xi):
        xi0 = xi[0]
        xi1 = xi[1]
        xi2 = xi[2]
        return jnp.array([[  -(1.0 - xi1)*(1.0 - xi2)/8.0, -(1.0 - xi0)*(1.0 - xi2)/8.0, -(1.0 - xi0)*(1.0 - xi1)/8.0],
                         [   (1.0 - xi1)*(1.0 - xi2)/8.0, -(1.0 + xi0)*(1.0 - xi2)/8.0, -(1.0 + xi0)*(1.0 - xi1)/8.0],
                         [   (1.0 + xi1)*(1.0 - xi2)/8.0,  (1.0 + xi0)*(1.0 - xi2)/8.0, -(1.0 + xi0)*(1.0 + xi1)/8.0],
                         [  -(1.0 + xi1)*(1.0 - xi2)/8.0,  (1.0 - xi0)*(1.0 - xi2)/8.0, -(1.0 - xi0)*(1.0 + xi1)/8.0],
                         [  -(1.0 - xi1)*(1.0 + xi2)/8.0, -(1.0 - xi0)*(1.0 + xi2)/8.0,  (1.0 - xi0)*(1.0 - xi1)/8.0],
                         [   (1.0 - xi1)*(1.0 + xi2)/8.0, -(1.0 + xi0)*(1.0 + xi2)/8.0,  (1.0 + xi0)*(1.0 - xi1)/8.0],
                         [   (1.0 + xi1)*(1.0 + xi2)/8.0,  (1.0 + xi0)*(1.0 + xi2)/8.0,  (1.0 + xi0)*(1.0 + xi1)/8.0],
                         [  -(1.0 + xi1)*(1.0 + xi2)/8.0,  (1.0 - xi0)*(1.0 + xi2)/8.0,  (1.0 - xi0)*(1.0 + xi1)/8.0],
                         ]) 

    def der_X_xi(self, xi):  # 7.6b
        return jnp.einsum('...ai,aj', self.nodes, jax.jit(self.der_N_fun)(xi))

    def der_N_X(self, xi):  # 7.6b
        temp = self.der_X_xi(xi).transpose(0,2,1)
        inv_der_X_xi = np.linalg.inv(temp)
        out = np.matmul(inv_der_X_xi, self.der_N_fun(xi).T).transpose(0,2,1)
        ##print(out.shape)
        return out

    def der_x_xi(self, x, xi):  # 7.11a
        return jnp.einsum('...ai,...aj', x, jax.jit(self.der_N_fun)(xi))

    def der_N_x(self, x, xi):  # 7.11b
        temp = self.der_x_xi(x, xi).transpose(0,2,1)
        inv_der_x_xi = jnp.linalg.inv(temp)
        
        return jnp.matmul(inv_der_x_xi,jax.jit(self.der_N_fun)(xi).T).transpose(0,2,1)
    
    def f_gauss(self, x_n):  # gradiente de deformacion -- 7.5
        #print("disp", x_n)
        x = self._get_nodes(x_n)
        #print(f"xn: {x}")
        #print(self.der_N_X_gp)
        F = jnp.einsum('eai,exaj->exij', x, self.der_N_X_gp)
        #print("test", F)
        return F
    
    def f(self, x_n):  # gradiente de deformacion -- 7.5
        #print("disp", x_n)
        x = self._get_nodes(x_n)
        Fs = []
        
        F = np.einsum('eai,exaj->exij', x, self.der_N_X_esquinas)
    
        #print("test", F)
        return F
    
    def Delphino(self,F,c1,c2):
        F_T = jnp.moveaxis(F,-1,-2)
        C = jnp.matmul(F_T,F)
        I1,I2,I3 = jnp.einsum('...ii',C), 0.5 * (jnp.einsum('...ii',C)**2 - jnp.einsum('...ii',jnp.matmul(C,C))), jnp.linalg.det(C)
        aux = I3**(-1/3)
        energia  =  (c1/c2) * (jnp.e**(c2*0.5*(I1*aux-3)) - 1 )    
        #print(energia)    
        return energia


    def Delphino_E(self,x_n,c1,c2):
        # x_n nuevas coordenadas
        #print(x_n)
        x = jax.jit(self._get_nodes)(x_n)
        F = jax.jit(self.f_gauss)(x_n)
        # print(F.shape)
        temp = self.Delphino(F,c1,c2)
        # print(temp.shape)
        micro = []
        for it, gp in enumerate(self.gauss_points):
            #print(x.shape)
            aux = jnp.linalg.det(jax.jit(self.der_x_xi)(x, gp))
            micro.append(aux)
        e_t = jnp.dot(temp,jnp.array(micro))
        # print(jnp.array(micro).sum())
        # # print(e_t)
        # #print(micro)
        # print(x_n)
        return e_t

In [11]:
cubo = energy(mesh.points, mesh.cells_dict[12])

cubo.Delphino_E(mesh_def.points, 30.0E-3,3.77)

Array([[7.9306464]], dtype=float32)

In [10]:
cubo.f(mesh.points)

array([[[[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]]])